## Import libraries

In [6]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

## Import files

In [7]:
horrible50 = pd.read_csv('../data/clean/WS_50horriblealbums_clean.csv', encoding='latin1')
horrible50.head(2)

,rshorriblerankingorder,rsalbum,rsartist,rsrationale,rsyear
0,50,Its Hard,The Who,"In the early Eighties, Pete Townshend was jugg...",1982
1,49,The Bridge,Billy Joel,Billy Joel had nearly a solid decade of succes...,1986


In [10]:
horrible50.rename(columns={'rsalbum': 'albumnamers'}, inplace=True)
horrible50.columns

Index(['rshorriblerankingorder', 'albumnamers', 'rsartist', 'rsrationale',
       'rsyear'],
      dtype='object')

In [11]:
spotifyalbums = pd.read_csv('../data/clean/api_albums_clean.csv', encoding='latin1')
spotifyalbums.head(3)

,albumnamers,albumidspotify,albumnamespotify,artistnamespotify,artistidspotify,releasedate,totaltracks
0,Sgt. Pepper's Lonely Hearts Club Band,6QaVfG1pHYl1z15ZxkvVDW,Sgt. Pepper's Lonely Hearts Club Band (Remaste...,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,1967-06-01,13.0
1,Pet Sounds,6GphKx2QAPRoVGWE9D7ou8,Pet Sounds (Original Mono & Stereo Mix),The Beach Boys,3oDbviiivRWhXwIE8hxkVV,1966-06-16,27.0
2,Revolver,3PRoXYsngSwjEQWR5PsHWR,Revolver (Remastered),The Beatles,3WrFJ7ztbogyGnTHbHJFl2,1966-08-05,14.0


## Merge tables

In [12]:
sqltable_horrible50 = pd.merge(horrible50, spotifyalbums[['albumnamers', 'albumidspotify']], how='left', on='albumnamers')

In [13]:
sqltable_horrible50.head(2)

,rshorriblerankingorder,albumnamers,rsartist,rsrationale,rsyear,albumidspotify
0,50,Its Hard,The Who,"In the early Eighties, Pete Townshend was jugg...",1982,46rjSfrOaPBPX8npn50l0A
1,49,The Bridge,Billy Joel,Billy Joel had nearly a solid decade of succes...,1986,2fRxSC6FtiAkhEDVZr2seH


In [14]:
sqltable_horrible50.shape

(50, 6)

## Clean NAs and duplicates

In [15]:
#clean NAs (some albums available in the RS DB were not found in Spotify)
sqltable_horrible50.dropna(inplace=True)

In [16]:
sqltable_horrible50.shape

(36, 6)

In [17]:
sqltable_horrible50.duplicated().any()

False

## Column names

In [18]:
sqltable_horrible50.columns

Index(['rshorriblerankingorder', 'albumnamers', 'rsartist', 'rsrationale',
       'rsyear', 'albumidspotify'],
      dtype='object')

In [19]:
finalcolumns = ['rs_horriblerankingorder', 'rs_albumname', 'rs_artist', 'rationale', 'rs_year','sp_albumid']

In [20]:
sqltable_horrible50.columns = finalcolumns

In [21]:
sqltable_horrible50.head(1)

,rs_horriblerankingorder,rs_albumname,rs_artist,rationale,rs_year,sp_albumid
0,50,Its Hard,The Who,"In the early Eighties, Pete Townshend was jugg...",1982,46rjSfrOaPBPX8npn50l0A


## Reoder columns

In [22]:
sqltable_horrible50 = sqltable_horrible50.iloc[:, [5, 0, 1, 2, 4]]
sqltable_horrible50.head(1)

,sp_albumid,rs_horriblerankingorder,rs_albumname,rs_artist,rs_year
0,46rjSfrOaPBPX8npn50l0A,50,Its Hard,The Who,1982


In [23]:
sqltable_horrible50['album_sp_albumid'] = sqltable_horrible50['sp_albumid']

In [24]:
sqltable_horrible50.head(1)

,sp_albumid,rs_horriblerankingorder,rs_albumname,rs_artist,rs_year,album_sp_albumid
0,46rjSfrOaPBPX8npn50l0A,50,Its Hard,The Who,1982,46rjSfrOaPBPX8npn50l0A


## Data type

In [25]:
sqltable_horrible50.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 49
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   sp_albumid               36 non-null     object
 1   rs_horriblerankingorder  36 non-null     int64 
 2   rs_albumname             36 non-null     object
 3   rs_artist                36 non-null     object
 4   rs_year                  36 non-null     int64 
 5   album_sp_albumid         36 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.0+ KB


In [26]:
sqltable_horrible50['rs_year'] = pd.to_datetime(sqltable_horrible50['rs_year'], format='%Y')

In [27]:
sqltable_horrible50['rs_year'].dtype

dtype('<M8[ns]')

In [28]:
sqltable_horrible50.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 49
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   sp_albumid               36 non-null     object        
 1   rs_horriblerankingorder  36 non-null     int64         
 2   rs_albumname             36 non-null     object        
 3   rs_artist                36 non-null     object        
 4   rs_year                  36 non-null     datetime64[ns]
 5   album_sp_albumid         36 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 2.0+ KB


## Export 

In [30]:
sqltable_horrible50.to_csv('../data/tableau/tableautable_horrible50.csv', index=True, encoding='latin1')